In [1]:
#Load fact table to a frame, get the record count
df=spark.read.format("parquet").load("Files/raw/fact_sale_1y_full/*.parquet")
df.count()

StatementMeta(, b30db958-81d1-4b3c-975f-af9f3ef8d2cb, 3, Finished, Available)

21728643

In [1]:
#Write Facts to a table in 21 sec!
df.write.mode("overwrite").format("delta").save("Tables/fact_sale")

StatementMeta(, ac6d805c-d5d6-43cd-8986-1947b4b0f832, 3, Finished, Available)

NameError: name 'df' is not defined

In [3]:
#Load Dimensions
from pyspark.sql.types import *
def loadFullDataFromSource(table_name):
    df = spark.read.format("parquet").load('Files/raw/' + table_name)
    df.write.mode("overwrite").format("delta").save("Tables/" + table_name)
full_tables = [
    'dimension_city',
    'dimension_date',
    'dimension_employee',
    'dimension_stock_item'
    ]
for table in full_tables:
    loadFullDataFromSource(table)

StatementMeta(, b30db958-81d1-4b3c-975f-af9f3ef8d2cb, 5, Finished, Available)

In [4]:
df_fact_sale = spark.read.table("wwiDW.fact_sale") 
df_dimension_date = spark.read.table("wwiDW.dimension_date")
df_dimension_city = spark.read.table("wwiDW.dimension_city")
sale_by_date_city = df_fact_sale.alias("sale") \
.join(df_dimension_date.alias("date"), df_fact_sale.InvoiceDateKey == df_dimension_date.Date, "inner") \
.join(df_dimension_city.alias("city"), df_fact_sale.CityKey == df_dimension_city.CityKey, "inner") \
.select("date.Date", "date.CalendarMonthLabel", "date.Day", "date.ShortMonth", "date.CalendarYear", "city.City", "city.StateProvince", "city.SalesTerritory", "sale.TotalExcludingTax", "sale.TaxAmount", "sale.TotalIncludingTax", "sale.Profit")\
.groupBy("date.Date", "date.CalendarMonthLabel", "date.Day", "date.ShortMonth", "date.CalendarYear", "city.City", "city.StateProvince", "city.SalesTerritory")\
.sum("sale.TotalExcludingTax", "sale.TaxAmount", "sale.TotalIncludingTax", "sale.Profit")\
.withColumnRenamed("sum(TotalExcludingTax)", "SumOfTotalExcludingTax")\
.withColumnRenamed("sum(TaxAmount)", "SumOfTaxAmount")\
.withColumnRenamed("sum(TotalIncludingTax)", "SumOfTotalIncludingTax")\
.withColumnRenamed("sum(Profit)", "SumOfProfit")\
.orderBy("date.Date", "city.StateProvince", "city.City")
sale_by_date_city.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/aggregate_sale_by_date_city")


StatementMeta(, b30db958-81d1-4b3c-975f-af9f3ef8d2cb, 6, Finished, Available)